In [5]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
import math
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools
import joblib


In [4]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [6]:
datapath ='/home02/chenhuangrong/RFH_feature.csv'
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
Y.extend(Y2)
Y = np.array(Y)
svc = svm.SVC()
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
clf = GridSearchCV(svc, parameters, cv=10, n_jobs=-1, scoring='accuracy')
clf.fit(X, Y)
C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
print clf.best_score_
gamma=clf.best_params_['gamma']
y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,),X,Y,cv=10,n_jobs=-1)
print type(y_predict)
y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/blending_files/RFH_predict.csv',header=None,index=False)
ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
#     easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,'/home02/chenhuangrong/cross_validation_frequency_%d_gram.xls'%(gram))
return ACC

0.734889058914
<type 'numpy.ndarray'>
[[['svmC:0.561231024155gamma:0.03125', 0.7348890589135425, 0.7376160990712074, 0.7291507268553941, 0.7291507268553941, 0.740627390971691, 0.7348666548810217, 0.73335898422470169, 0.73335898422470169, 0.46980905901870224, '0.7348', 953.0, 354.0, 339.0, 968.0, 1307.0, 1307.0]]]


SyntaxError: 'return' outside function (<ipython-input-6-0b3e4d036687>, line 33)